In [1]:
import warnings
warnings.filterwarnings('ignore')
# from datetime import datetime, date
import numpy as np
import pandas as pd
# import yfinance as yf

In [2]:
# A empresa e Indicadores fundamentalistas dados do statusinvest.com.br
#(pode ter atraso, dados não são pegos diretamente no SI)

url = 'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/SI_Acoes.csv'
pd.set_option('display.max_columns', None)
fundsSI = pd.read_csv(url,sep=';' , decimal=',' ,thousands ='.' )

In [3]:
fundsSI.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [4]:
fundsSI =  fundsSI[fundsSI[' LIQUIDEZ MEDIA DIARIA'] > 300000] 
fundsSI =  fundsSI[(fundsSI[' LPA'] > 0) & (fundsSI[' VPA'] > 0) ]
# fundsSI

In [5]:
fundsSI['valor_intrinseco'] = np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA'])
fundsSI['Delta'] = fundsSI['valor_intrinseco'] / fundsSI['PRECO'] -1
fundsSI["Rank"]   = fundsSI['Delta'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
1,LUPA3,2.04,8.714987,191.0,3.272052,0.87,3.88
2,GOAU4,9.87,32.252023,190.0,2.267682,2.66,17.38
3,GOAU3,10.16,32.252023,189.0,2.174412,2.66,17.38
4,HBOR3,2.82,8.386596,188.0,1.973970,0.30,10.42
5,HBRE3,5.16,14.570089,187.0,1.823661,0.51,18.50
6,GGBR3,19.61,54.774447,186.0,1.793190,4.63,28.80
7,MEAL3,2.20,5.982725,185.0,1.719421,0.41,3.88
8,GGBR4,21.23,54.774447,184.0,1.580049,4.63,28.80
9,BRAP3,19.88,49.797741,183.0,1.504917,5.40,20.41
10,TECN3,3.88,9.446904,182.0,1.434769,0.74,5.36


In [6]:
fundsSI =  fundsSI[(fundsSI['P/L'] > 0)]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
1,LUPA3,2.04,8.714987,191.0,3.272052,0.87,3.88
2,GOAU4,9.87,32.252023,190.0,2.267682,2.66,17.38
3,GOAU3,10.16,32.252023,189.0,2.174412,2.66,17.38
4,HBOR3,2.82,8.386596,188.0,1.973970,0.30,10.42
5,HBRE3,5.16,14.570089,187.0,1.823661,0.51,18.50
6,GGBR3,19.61,54.774447,186.0,1.793190,4.63,28.80
7,MEAL3,2.20,5.982725,185.0,1.719421,0.41,3.88
8,GGBR4,21.23,54.774447,184.0,1.580049,4.63,28.80
9,BRAP3,19.88,49.797741,183.0,1.504917,5.40,20.41
10,TECN3,3.88,9.446904,182.0,1.434769,0.74,5.36


In [7]:
fundsSI =  fundsSI[(fundsSI['CAGR RECEITAS 5 ANOS'] > 0) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
2,GOAU4,9.87,32.252023,190.0,2.267682,2.66,17.38
3,GOAU3,10.16,32.252023,189.0,2.174412,2.66,17.38
4,HBOR3,2.82,8.386596,188.0,1.973970,0.30,10.42
5,HBRE3,5.16,14.570089,187.0,1.823661,0.51,18.50
6,GGBR3,19.61,54.774447,186.0,1.793190,4.63,28.80
7,MEAL3,2.20,5.982725,185.0,1.719421,0.41,3.88
8,GGBR4,21.23,54.774447,184.0,1.580049,4.63,28.80
10,TECN3,3.88,9.446904,182.0,1.434769,0.74,5.36
12,SAPR3,4.92,11.827383,180.0,1.403940,0.99,6.28
14,CMIG4,11.72,27.554310,178.0,1.351050,3.04,11.10


In [8]:
fundsSI =  fundsSI[(fundsSI['CAGR LUCROS 5 ANOS'] > 0) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
7,MEAL3,2.20,5.982725,185.0,1.719421,0.41,3.88
12,SAPR3,4.92,11.827383,180.0,1.403940,0.99,6.28
14,CMIG4,11.72,27.554310,178.0,1.351050,3.04,11.10
17,NTCO3,16.98,39.493123,175.0,1.325861,3.43,20.21
18,COCE5,40.00,92.970298,174.0,1.324257,7.05,54.49
19,SAPR11,25.48,59.036120,173.0,1.316959,4.93,31.42
20,ALOS3,24.24,56.101845,172.0,1.314433,5.46,25.62
21,SAPR4,5.13,11.827383,171.0,1.305533,0.99,6.28
23,NEOE3,21.33,48.354565,169.0,1.266974,4.28,24.28
24,EUCA4,17.78,39.888636,168.0,1.243455,2.97,23.81


In [9]:
fundsSI =  fundsSI[(fundsSI['DIVIDA LIQUIDA / EBIT'] <4) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
7,MEAL3,2.20,5.982725,185.0,1.719421,0.41,3.88
12,SAPR3,4.92,11.827383,180.0,1.403940,0.99,6.28
14,CMIG4,11.72,27.554310,178.0,1.351050,3.04,11.10
17,NTCO3,16.98,39.493123,175.0,1.325861,3.43,20.21
19,SAPR11,25.48,59.036120,173.0,1.316959,4.93,31.42
20,ALOS3,24.24,56.101845,172.0,1.314433,5.46,25.62
21,SAPR4,5.13,11.827383,171.0,1.305533,0.99,6.28
23,NEOE3,21.33,48.354565,169.0,1.266974,4.28,24.28
24,EUCA4,17.78,39.888636,168.0,1.243455,2.97,23.81
26,ROMI3,12.91,27.447787,166.0,1.126087,2.62,12.78


In [10]:
# linhas_com_nan = top20[top20.isna().any(axis=1)]
# linhas_com_nan

### Porque 22,5?

O número 22,5 na Fórmula de Graham é um fator de ponderação que tem um propósito específico. Essa constante foi escolhida por Graham pra ajustar a avaliação do preço justo de uma ação com base na taxa de crescimento anual esperada da empresa.

O número 22,5 é, na verdade, o resultado da multiplicação de 8,5 por 2,65 (8,5 x 2,65 = 22,5). O número 8,5 é a base que Graham considerou razoável pra uma empresa com taxa de crescimento zero, ou seja, uma empresa que não cresce.

Já o número 2,65 representa a média do retorno exigido pelos investidores no mercado de ações durante a época de Graham, que era de aproximadamente 4,4% acima da taxa de retorno dos títulos do Tesouro dos Estados Unidos.

O fator 22,5 ajuda a ajustar o preço justo com base no crescimento da empresa e na expectativa de retorno dos investidores.

Esse ajuste garante que a Fórmula de Graham considere a taxa de crescimento anual esperada e reflita uma avaliação mais realista do preço justo de uma ação.

No próximo tópico, abordaremos a calculadora do preço justo usando a Fórmula de Graham.